## Predicting Fantasy Football Draftability Demo

## Importing necessary libraries

In [ ]:
# Rerun this code cell until all libraries are imported successfully
try:
  import gradio as gr   # https://www.gradio.app/docs
  import pandas as pd   # https://pandas.pydata.org/docs/
  import numpy as np   # https://numpy.org/doc/
  from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer   # https://scikit-learn.org/stable/modules/preprocessing.html
  import pickle   # https://docs.python.org/3/library/pickle.html
  print("All libraries have been imported successfully...")
except:
  !pip install gradio
  !pip install pandas
  !pip install numpy
  !pip install scikit-learn
  !pip install pickle
  print("Some libraries are not found, installing...")

All libraries have been imported successfully...


# Creating the demos

## Demo using manual inputs

In [ ]:
# Load the pre-trained model
with open("Random_Forest.pkl", "rb") as model_file:
  best_model = pickle.load(model_file)

# Define numerical and position columns
numerical_columns = ["Age", "G", "GS", "Cmp", "Att", "Yds", "TD", "Int",
                     "RushAtt", "RushYds", "YA", "RushTD", "Tgt", "Rec",
                     "RecYds", "YR", "RecTD", "Fmb", "FL", "PPR"]
name_columns = ["Age", "G - Games Played", "GS - Games Started", "Cmp - Completed Passes", "Att - Attempts", "Yds - Total Yards", "TD - Touchdowns", "Int - Interceptions",
                     "RushAtt - Rushing Attempts", "RushYds - Rushing Yards", "YA - Yards After Catch", "RushTD - Rushing Touchdowns", "Tgt - Targets", "Rec - Receptions",
                     "RecYds - Receiving Yards", "YR - Yards per Reception", "RecTD - Receiving Touchdowns", "Fmb - Fumbles", "FL - Fumbles Lost", "PPR - Points per Reception"]
positions = ["Quarterback", "Running Back", "Tight End", "Wide Receiver"]
encoded_positions = [f"FantPos_{pos}" for pos in positions]  # ["FantPos_QB", "FantPos_RB", ...]

# Fit preprocessors
onehot_encoder = OneHotEncoder(sparse_output=False).fit(np.array(positions).reshape(-1, 1))

# Prediction function
def predict_fantasy(*inputs):
  # Replacing all empty values with 0 or a default value
  full_inputs = [0 if x is None else x for x in inputs[:-1]]
  if inputs[-1] is None:
    full_inputs.append("QB")
  else:
    full_inputs.append(inputs[-1])
  inputs = full_inputs

  try:
    # Separate numerical inputs and position
    numerical_inputs = np.array(inputs[:-1]).reshape(1, -1)
    position_input = inputs[-1]

    # Encode position as "FantPos_*"
    position_encoded = np.zeros(len(encoded_positions))
    position_encoded[positions.index(position_input)] = 1
    position_encoded = position_encoded.reshape(1, -1)  # Ensure it's 2D

    # Combine scaled numerical inputs and encoded position
    combined_inputs = np.hstack([numerical_inputs, position_encoded])

    # Predict using the model
    prediction = best_model.predict(combined_inputs)
    print(combined_inputs)
    return prediction[0]
  except Exception as e:
    return f"Error: {str(e)}"

# Gradio Interface
feature_inputs = [gr.Number(label=namecol) for col, namecol in zip(numerical_columns, name_columns)] + [gr.Dropdown(choices=positions, label="Position")]
demo = gr.Interface(fn=predict_fantasy,
                    inputs=feature_inputs,
                    outputs=[gr.Number(label="Position Ranking")],
                    title="Fantasy Football Rank Predictor",
                    description="Provide key player stats and position to predict their positional rank.")

# Launch the Gradio demo
demo.launch(debug=False)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98c4a2958d2b38c369.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Demo using sliders

In [ ]:
# Load the pre-trained model
with open("Random_Forest.pkl", "rb") as model_file:
  best_model = pickle.load(model_file)

# Define numerical and position columns
numerical_columns = ["Age", "G", "GS", "Cmp", "Att", "Yds", "TD", "Int",
                     "RushAtt", "RushYds", "YA", "RushTD", "Tgt", "Rec",
                     "RecYds", "YR", "RecTD", "Fmb", "FL", "PPR"]
name_columns = ["Age", "G - Games Played", "GS - Games Started", "Cmp - Completed Passes", "Att - Attempts", "Yds - Total Yards", "TD - Touchdowns", "Int - Interceptions",
                     "RushAtt - Rushing Attempts", "RushYds - Rushing Yards", "YA - Yards per Attempt", "RushTD - Rushing Touchdowns", "Tgt - Targets", "Rec - Receptions",
                     "RecYds - Receiving Yards", "YR - Yards per Reception", "RecTD - Receiving Touchdowns", "Fmb - Fumbles", "FL - Fumbles Lost", "PPR - Points per Reception"]
positions = ["Quarterback", "Running Back", "Tight End", "Wide Receiver"]
encoded_positions = [f"FantPos_{pos}" for pos in positions]  # ["FantPos_QB", "FantPos_RB", ...]

# Fit preprocessors
onehot_encoder = OneHotEncoder(sparse_output=False).fit(np.array(positions).reshape(-1, 1))

# Prediction function
def predict_fantasy(*inputs):
  print(inputs)
  # Replacing all empty values with 0 or a default value
  full_inputs = [0 if x is None else x for x in inputs[:-1]]
  if inputs[-1] is None:
    full_inputs.append("QB")
  else:
    full_inputs.append(inputs[-1])
  inputs = full_inputs

  try:
    # Separate numerical inputs and position
    numerical_inputs = np.array(inputs[:-1]).reshape(1, -1)
    position_input = inputs[-1]

    # Encode position as "FantPos_*"
    position_encoded = np.zeros(len(encoded_positions))
    position_encoded[positions.index(position_input)] = 1
    position_encoded = position_encoded.reshape(1, -1)  # Ensure it's 2D

    # Combine scaled numerical inputs and encoded position
    combined_inputs = np.hstack([numerical_inputs, position_encoded])

    # Predict using the model
    prediction = best_model.predict(combined_inputs)
    print(combined_inputs)
    return prediction[0]
  except Exception as e:
    return f"Error: {str(e)}"

data_df = pd.read_csv("Fantasy_Football_Stats.csv", encoding='latin-1')

# Gradio Interface
feature_inputs = [gr.Slider(0, (((data_df[col].max() + 99) // 100) * 100), step=1, label=namecol, interactive=True) for col, namecol in zip(numerical_columns, name_columns)] + [gr.Dropdown(choices=positions, label="Position")]
demo = gr.Interface(fn=predict_fantasy,
                    inputs=feature_inputs,
                    outputs=[gr.Number(label="Position Ranking")],
                    title="Fantasy Football Rank Predictor",
                    description="Provide key player stats and position to predict their positional rank.")

# Launch the Gradio demo
demo.launch(debug=False)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84f6e2bd19d55bc4ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
